In [3]:
import requests
import json
from bs4 import BeautifulSoup
from urllib import parse

# 1. Comic List 가져오기

In [4]:
comic_resp = requests.get('https://comic.naver.com/webtoon/weekday.nhn')


In [5]:
soup = BeautifulSoup(comic_resp.content)


In [6]:
comic_soup_list = soup.select('.col>.col_inner ul li')
comic_soup_list[0]

<li>
<div class="thumb">
<a href="/webtoon/list.nhn?titleId=183559&amp;weekday=mon" onclick="nclk_v2(event,'thm*m.img','','1')">
<img alt="신의 탑" height="90" onerror="this.src='https://static-comic.pstatic.net/staticImages/COMICWEB/NAVER/img/common/blank.gif'" src="https://shared-comic.pstatic.net/thumb/webtoon/183559/thumbnail/thumbnail_IMAG10_5e13c29c-f451-4430-a84a-a46495fb8cc3.jpg" title="신의 탑" width="83"/><span class="mask"></span>
</a>
</div>
<a class="title" href="/webtoon/list.nhn?titleId=183559&amp;weekday=mon" onclick="nclk_v2(event,'thm*m.tit','','1')" title="신의 탑">신의 탑</a>
</li>

In [7]:
def get_param_from_href(href, param):
    url = parse.urlparse(href)
    url_query = parse.parse_qs(url.query)
    return url_query[param][0]
    

In [8]:
comic_dict = {}
for comic_li in comic_soup_list:
    link_url = comic_li.a['href']
    title_id = get_param_from_href(link_url, 'titleId')

    title = comic_li.img['title']
    comic_dict[title] = {
        'title_id': title_id,
        'title': title,
        'link': link_url,
    }

# 2. Comic 마지막 화 번호 가져오기

In [9]:
BASE_URL = 'https://comic.naver.com'
for idx, comic in enumerate(comic_dict):
    resp = requests.get(BASE_URL + comic_dict[comic]['link'])
    soup = BeautifulSoup(resp.content)
    
    recent_href = soup.select_one('table.viewList>tr>td.title>a')['href']
    recent_no = get_param_from_href(recent_href, 'no')
    comic_dict[comic]['recent_no'] = recent_no
    print("{}이 진행중입니다. {}/{}".format(comic, idx, len(comic_dict)))


신의 탑이 진행중입니다. 0/237
뷰티풀 군바리이 진행중입니다. 1/237
윈드브레이커이 진행중입니다. 2/237
소녀의 세계이 진행중입니다. 3/237
평범한 8반이 진행중입니다. 4/237
데드라이프이 진행중입니다. 5/237
니편내편이 진행중입니다. 6/237
용비불패 완전판이 진행중입니다. 7/237
가우스전자 시즌3~4이 진행중입니다. 8/237
위대한 방옥숙이 진행중입니다. 9/237
링크보이이 진행중입니다. 10/237
첫사랑입니다만이 진행중입니다. 11/237
치즈인더트랩 (재)이 진행중입니다. 12/237
신을 죽이는 방법이 진행중입니다. 13/237
마왕이 되는 중2야이 진행중입니다. 14/237
이것도 친구라고이 진행중입니다. 15/237
내 여자친구는 상남자이 진행중입니다. 16/237
유일무이 로맨스이 진행중입니다. 17/237
푸들과 Dog거중이 진행중입니다. 18/237
일진이 사나워이 진행중입니다. 19/237
히어로메이커이 진행중입니다. 20/237
오늘의 순정망화이 진행중입니다. 21/237
요괴대전이 진행중입니다. 22/237
부로콜리왕자이 진행중입니다. 23/237
귀인이 진행중입니다. 24/237
삶이 우리를 속일지라도이 진행중입니다. 25/237
같은도장이 진행중입니다. 26/237
피플이 진행중입니다. 27/237
꿈의 기업이 진행중입니다. 28/237
장씨세가 호위무사이 진행중입니다. 29/237
닥터 하운드이 진행중입니다. 30/237
내 어린고양이와 늙은개 (재)이 진행중입니다. 31/237
오직 나의 주인님이 진행중입니다. 32/237
하늘쌤은 피곤해이 진행중입니다. 33/237
교환일기이 진행중입니다. 34/237
플랫다이어리이 진행중입니다. 35/237
뱀파이어의 꽃이 진행중입니다. 36/237
조선팔도 최강아이돌이 진행중입니다. 37/237
여신강림이 진행중입니다. 38/237
마음의소리이 진행중입니다. 39/237
랜덤채팅의 그녀!이 진행중입니다. 40/237
덴마이 진행중입니다. 41/237
사이드킥 

# Comic Comment가져오기 (이름으로 & (title_id, comic_no)

In [10]:
comment_url = "https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json"

In [11]:
comic_name = '신의 탑'
title_id = comic_dict[comic_name]['title_id']
comic_no = comic_dict[comic_name]['recent_no']

# title_id = 183559
# comic_no = 432

In [12]:
params = {
    'ticket':'comic',
    'templateId':'webtoon',
    'pool':'cbox3',
#     'callback':'jQuery112406956169591847585_1559905255662',
    'lang': 'ko',
    'country':'KR',
    'objectId': '{}_{}'.format(title_id,comic_no),
    'pageSize': 15,
    'indexSize': 10,
#     'listType':'OBJECT',
    'pageType': 'default',
    'page':1,
#     'initialize':'true',
#     'useAltSort':'true',
    'replyPageSize':10,
#     '_':1559905255663
}

headers = {
    'Referer': 'https://comic.naver.com/comment/comment.nhn?titleId={}&no={}'.format(title_id, comic_no)
}

In [14]:
resp = requests.get(comment_url, headers=headers, params=params)

In [15]:
callback_content = resp.content

In [16]:
content = callback_content[10:-2]
content

b'{"success":true,"code":"1000","message":"\xec\x9a\x94\xec\xb2\xad\xec\x9d\x84 \xec\x84\xb1\xea\xb3\xb5\xec\xa0\x81\xec\x9c\xbc\xeb\xa1\x9c \xec\xb2\x98\xeb\xa6\xac\xed\x95\x98\xec\x98\x80\xec\x8a\xb5\xeb\x8b\x88\xeb\x8b\xa4.","lang":"ko","country":"KR","result":{"commentList":[{"ticket":"comic","objectId":"183559_432","categoryId":"*","templateId":"default","commentNo":370475357,"parentCommentNo":370475357,"replyLevel":1,"replyCount":0,"replyAllCount":0,"replyPreviewNo":null,"replyList":null,"imageCount":0,"imageList":null,"imagePathList":null,"imageWidthList":null,"imageHeightList":null,"commentType":"txt","stickerId":null,"sticker":null,"sortValue":1557671637545,"contents":"\xeb\xb0\xa4: \xeb\xaa\xb8\xec\x97\x90 \xea\xb3\x84\xec\x86\x8d \xeb\xad\x94\xea\xb0\x88 \xec\xb0\x94\xeb\x9f\xac\xeb\x84\xa3\xec\x9c\xbc\xeb\x8b\x88\xea\xb9\x8c \xea\xb0\x95\xed\x95\xb4\xec\xa7\x80\xeb\x8d\x94\xeb\x9d\xbc\xea\xb5\xac\xec\x9a\x94. \xec\xbf\xa4: \xea\xb3\x84\xec\x86\x8d \xec\xa3\xbd\xec\x96\xb4\x

In [17]:
import json

In [18]:
json_content = json.loads(content)
json_content

{'success': True,
 'code': '1000',
 'message': '요청을 성공적으로 처리하였습니다.',
 'lang': 'ko',
 'country': 'KR',
 'result': {'commentList': [{'ticket': 'comic',
    'objectId': '183559_432',
    'categoryId': '*',
    'templateId': 'default',
    'commentNo': 370475357,
    'parentCommentNo': 370475357,
    'replyLevel': 1,
    'replyCount': 0,
    'replyAllCount': 0,
    'replyPreviewNo': None,
    'replyList': None,
    'imageCount': 0,
    'imageList': None,
    'imagePathList': None,
    'imageWidthList': None,
    'imageHeightList': None,
    'commentType': 'txt',
    'stickerId': None,
    'sticker': None,
    'sortValue': 1557671637545,
    'contents': '밤: 몸에 계속 뭔갈 찔러넣으니까 강해지더라구요. 쿤: 계속 죽어보니까 강해지더라고',
    'userIdNo': '3J0f9',
    'exposedUserIp': None,
    'lang': 'ko',
    'country': 'CN',
    'idType': 'naver',
    'idProvider': 'naver',
    'userName': '노이하우스',
    'userProfileImage': 'https://profile-phinf.pstatic.net/47195/faad224449cd83b369e8e7b54d23bbcaf33f5cc254496d6c25076866de7812